In [51]:
import os
import requests
import requests
import ast

import pandas as pd
import glob
import plotly.express as px

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from time import sleep

from locations import *
from parsing import *

# LOCATIONS SCRAPING

In [52]:
# Base

base_url = 'https://www.pisos.com/'

venta_url = f'{base_url}venta/'

URL = 'https://www.pisos.com/viviendas/valencia/'

In [53]:
# Sacamos todas las paginas filtradas por provincias

response = requests.get(base_url)

soup = BeautifulSoup(response.text, "html.parser")

provincias_str = soup.find('div', class_ = 'home-container').find('div', class_ = 'selectBox').find_all('ul')[-1].text

provincias = [x.lower() for x in provincias_str.strip().split('\n')]

endpoints = replaceWithUnderscore(provincias)

endpoints = [venta_url + endpoint for endpoint in endpoints]

# with open('locations.txt', 'w') as file:
#     file.write('\n'.join(endpoints))

# SINGLE PAGE SCRAPING

In [54]:
def scrape(urls):
    browser = webdriver.Chrome()

    for idx, url in enumerate(urls):
        browser.get(url)
        # browser.maximize_window()

        if idx == 0:
            element = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
            )

            element.click() # Accept cookies

        # <SCROLLING>
        while True:
            is_at_bottom = browser.execute_script("return window.scrollY + window.innerHeight >= document.body.scrollHeight")
            browser.execute_script("window.scroll({ top: document.body.scrollHeight, behavior: 'smooth' });")
            if is_at_bottom:
                break
            sleep(0.2)
        # </SCROLLING>
        
        html_content = browser.page_source

        timestamp = ''.join(str(datetime.datetime.now().timestamp()).split('.'))

        soft_url = url.replace('https://www.pisos.com/comprar/', '')
        soft_url = soft_url.replace('/', '_')
        soft_url = soft_url.replace('-', '_')

        file_path = f'../html_content/{timestamp}_{soft_url}.html'
        try:
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(html_content)
        except Exception as e:
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(repr(e))

    browser.quit()

# TESTING

In [55]:
def scanRegions(url):
    response = requests.get(url)

    print(f'URL: {url} | STATUS {response.status_code}')

    soup = BeautifulSoup(response.text, 'html.parser')

    items = soup.select('div.zoneList a.item:not(.item-subitem)')

    endpoints = {}
    for item in items:
        endpoint = item['href']
        n_results = item.find('span', class_ = 'total').text

        # <INT>
        if len(n_results) != 0:
            try:
                n_results = n_results[1:-1]
                n_results = ''.join(n_results.split('.'))
                n_results = int(n_results)
            except:
                print(f'FAIL CASTING TO INTEGER {endpoint}')
        else:
            print(f'n_results EMPTY {endpoint}')
        #</INT>

        # <RECURSSION>
        if n_results > 3000:
            endpoints[endpoint] = scanRegions(base_url[:-1] + endpoint)
        # </RECURSSION>

        else: endpoints[endpoint] = n_results

    return endpoints

In [56]:
def parseRegions(endpoints):

    def extract(endpoints):
        array = []
        for key, value in endpoints.items():
            if isinstance(value, int):
                array.append(key)
                continue

            data = extract(value)
            array.extend(data)

        return array
    
    endpoints = extract(endpoints)

    array = []
    
    for endpoint in endpoints:
        data = endpoint
        if '/venta/pisos-' in endpoint:
            data = endpoint.replace('/venta/pisos-', '/viviendas/')
        array.append(data)

    return array

In [57]:
urls = scanRegions(URL)
urls = parseRegions(urls)
urls = [base_url + x for x in urls]
urls = [x.replace('//viviendas/', '/venta/pisos-') for x in urls]
print(urls)

URL: https://www.pisos.com/viviendas/albacete/ | STATUS 200
['https://www.pisos.com/venta/pisos-albacete_capital/', 'https://www.pisos.com/venta/pisos-campos_de_hellin/', 'https://www.pisos.com/venta/pisos-la_mancha_del_jucar_centro/', 'https://www.pisos.com/venta/pisos-la_manchuela/', 'https://www.pisos.com/venta/pisos-monte_iberico_corredor_de_almansa/', 'https://www.pisos.com/venta/pisos-sierra_de_alcaraz_y_campo_de_montiel/', 'https://www.pisos.com/venta/pisos-sierra_segura/']


In [58]:
def scrapeUrls(endpoint):
    response = requests.get(endpoint)

    soup = BeautifulSoup(response.text, 'html.parser')

    results = soup.find('div', class_ = 'grid__title').find_all('span')[-1].text

    n_results = int(''.join([x for x in results if x.isnumeric()]))
    n_pages = (n_results // 30) + 1

    urls = []
    for i in range(n_pages):
        url = f'{endpoint}{i + 1}'
        response = requests.get(url)
        timestamp = datetime.datetime.now().timestamp

        soup = BeautifulSoup(response.text, 'html.parser')

        ads = soup.find_all('a', class_ = 'ad-preview__title')
        urls.extend([x['href'] for x in ads])
    return urls

In [59]:
for url in urls:
    urls_ = scrapeUrls(url)
    urls_ = list(set(urls_))
    urls_ = [base_url[:-1] + x for x in urls_]
    with open('urls.csv', 'a+') as file:
        file.writelines([x + ',\n' for x in urls_])

In [60]:
with open('urls.csv') as file:
    urls = file.read()

urls = urls.split(',\n')

In [61]:
scrape(urls)

The chromedriver version (117.0.5938.92) detected in PATH at c:\Users\cuicuidev\Desktop\pisoscom\scraping\chromedriver.exe might not be compatible with the detected chrome version (118.0.5993.71); currently, chromedriver 118.0.5993.70 is recommended for chrome 118.*, so it is advised to delete the driver in PATH and retry


In [62]:
####################################################################################################################

NameError: name 'dalsjdjha' is not defined

In [ ]:
# Obtener la Data a partir de los archivos de la carpeta 'html_content'

data = {
    'price' : [],
    'title' : [],
    'location' : [],
    'lat' : [],
    'lng' : [],
    'characteristics' : [],
    'agency' : [],
    'updated' : []
}

files = glob.glob('html_content/*.html')

for file in files:
    with open(file, encoding='utf-8') as f:
        source = f.read()
    soup = BeautifulSoup(source, 'html.parser')

    price = getPrice(soup)
    title = getTitle(soup)
    location = getLocation(soup)
    lat, long = getLatLong(soup)
    characteristics = getCharacteristics(soup)
    updated, agency = getAgencyDate(soup)

    data['price'].append(price)
    data['title'].append(title)
    data['location'].append(location)
    data['lat'].append(lat)
    data['lng'].append(long)
    data['characteristics'].append(characteristics)
    data['agency'].append(agency)
    data['updated'].append(updated)

df = pd.DataFrame(data)

In [ ]:
# df.to_csv('madrid.csv')
df.to_csv('valencia.csv')

In [ ]:
####################################################################################################################

In [ ]:
df = pd.read_csv('madrid.csv')
df.drop('Unnamed: 0', axis = 1, inplace = True)

# DATA PROCESSING

In [ ]:
df['price'] = df['price'].map(lambda x: tryInt(x))
df['characteristics'] = df['characteristics'].apply(tryLiteralEval)

In [ ]:
df

,price,title,location,lat,lng,characteristics,agency,updated
0,176000.0,Apartamento en venta en Abantos-Monte Carmelo-...,Abantos-Monte Carmelo-El Rosario (San Lorenzo ...,40.593445,-4.145386,"[Superficie construida : 63 m², Superficie úti...",Inmuebles de SYMBOLIC GESTION INMOBILIARIA,1.674861e+09
1,116500.0,"Apartamento en venta en Calle de Ercilla, 20, ...","Calle de Ercilla, 20, cerca de Calle del Labra...",40.402079,-3.702151,"[Superficie construida : 25 m², Baños : 1, Pla...",Inmuebles de Inmobiliaria Grupo Habitazone,1.674947e+09
2,169000.0,Apartamento en venta en Ajalvir,Ajalvir,40.534457,-3.479415,"[Superficie construida : 70 m², Superficie úti...",Inmuebles de INMOBILIARIA TECNICA NORTE,1.673392e+09
3,174000.0,Apartamento en venta en Alcorcón,Casco Antiguo (Alcorcón),40.344358,-3.825283,"[Superficie construida : 60 m², Superficie úti...",Inmuebles de Elite Asesoramiento Inmobiliario SL,1.672960e+09
4,130000.0,Apartamento en venta en Alcorcón,Casco Antiguo (Alcorcón),40.347096,-3.827826,"[Superficie construida : 60 m², Superficie úti...",Inmuebles de ELEPHANT REAL ESTATE,1.672615e+09
...,...,...,...,...,...,...,...,...
15841,135000.0,Piso en venta en Calle de la Circunvalación,Calle de la Circunvalación. Zona Suroeste (Tor...,40.454606,-3.455234,"[Superficie construida : 67 m², Superficie úti...",Inmuebles de ANGEL & RICARDO INMOBILIARIA,1.672960e+09
15842,210000.0,Piso en venta en Calle de Urano,Calle de Urano. Zona Suroeste (Torrejón de Ardoz),40.441928,-3.473036,"[Superficie construida : 79 m², Superficie úti...",Inmuebles de Look & Find Retiro,1.672615e+09
15843,324600.0,Piso en venta en Fresnos,Zona Suroeste (Torrejón de Ardoz),40.448403,-3.470899,"[Superficie construida : 140 m², Habitaciones ...",Inmuebles de Cassas.es asesoria inmobiliaria,1.672874e+09
15844,249900.0,Piso en venta en Calle de Cibeles,Calle de Cibeles. Zona Suroeste (Torrejón de A...,40.444707,-3.473520,"[Superficie construida : 106 m², Superficie út...",Inmuebles de AS Fresnos Inmobiliaria,1.672874e+09


#### CHARACTERISTICS PARSING

In [ ]:
df_characteristics = freeChurro(df)

In [ ]:
df_ = pd.concat([df, df_characteristics], axis = 1).drop('characteristics', axis = 1)

In [ ]:
df_.to_csv('madrid_parsed.csv', sep = ",", index = False)

# NP.NAN

# CONCLUSION: HAY MUCHOS NANS

In [ ]:
df_1 = nanReport(0.3, df_)

Porcentaje de valores perdidos en total: 0.4556354916067147
Columnas conservadas: 14
